<a href="https://colab.research.google.com/github/SaiDeepa111/hello-world/blob/main/Copy_of_sumaya_multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
fakeddit = pd.read_csv('/content/drive/MyDrive/fakedit/multimodal_only_samples/multimodal_train.tsv',sep='\t')
val = pd.read_csv('/content/drive/MyDrive/fakedit/multimodal_only_samples/multimodal_validate.tsv',sep='\t')
test= pd.read_csv('/content/drive/MyDrive/fakedit/multimodal_only_samples/multimodal_test_public.tsv',sep='\t')

In [3]:
fakeddit=fakeddit[:500]

In [4]:
# Import required libraries
!pip install transformers
from transformers import RobertaTokenizer

# Load Roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Load dataset
# Assuming 'fakeddit' is a pandas DataFrame with a column named 'clean_title'
text_data = fakeddit['clean_title'].tolist()

# Clean text data (if needed)
# ... code to clean text data ...

# Tokenize text data
tokenized_text = []
for text in text_data:
    tokens = tokenizer.encode(text, add_special_tokens=True)
    tokenized_text.append(tokens)

# Encode text data
encoded_text = []
for tokens in tokenized_text:
    encoded = tokenizer.convert_tokens_to_ids(tokens)
    encoded_text.append(encoded)

# Pad text data to a fixed length (if needed)
max_length = max(len(tokens) for tokens in encoded_text)
max_length
padded_text = []
for encoded in encoded_text:
    padding = [0] * (max_length - len(encoded))
    padded = encoded + padding
    padded_text.append(padded)

# Convert text data to numpy array
import numpy as np
padded_text = np.array(padded_text)

# Now, 'padded_text' contains the preprocessed text data ready for input to the Roberta model


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.2 MB/s eta 0:00:00


In [5]:
# Import required libraries
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import urllib.request

# Load dataset
# Assuming 'fakeddit' is a pandas DataFrame with a column named 'image_url'
image_urls = fakeddit['image_url'].tolist()

# Load and preprocess image data
image_data = []
i = 0
for url in image_urls:
  try:
    # Download the image from the URL
    urllib.request.urlretrieve(url, f"image{i+1}.jpg")  # Save the image as "image{i+1}.jpg" in the local file system

    # Load the downloaded image using OpenCV
    image = cv2.imread(f"image{i+1}.jpg")  # Read the downloaded image from the local file system

    # Display the loaded image
    #cv2_imshow(image)
    #cv2.waitKey(0)

  except Exception as e:
    print(f"Failed to load image from URL {url}: {e}")

  # Resize image to a fixed size (e.g., 224x224) to match input size of ResNet
  if image is not None:
    # Perform resize operation
    new_size = (224, 224)  # Your desired output size
    resized_img = cv2.resize(image, new_size, interpolation=cv2.INTER_LINEAR)
    # Convert image to RGB format
    if resized_img is not None:
      resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
    # Convert image to BGR format
      resized_img = cv2.cvtColor(resized_img, cv2.COLOR_RGB2BGR)


    # Convert image to float32 and normalize pixel values to range [0, 1]
    if resized_img is not None:
      resized_img = resized_img.astype(np.float32) / 255.0

      # Perform data augmentation (if needed) using OpenCV or other image processing techniques
      # ... code for data augmentation ...

      # Add preprocessed image data to the list
      image_data.append(resized_img)
      i = i + 1

# Convert image data to numpy array
image_data = np.array(image_data, dtype=np.float32)

import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, image_data, labels):
        self.image_data = image_data
        self.labels = labels

    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, index):
        image = torch.from_numpy(self.image_data[index])
        label = torch.tensor(self.labels[index], dtype=torch.long)
        return image, label


Failed to load image from URL https://preview.redd.it/wsfx0gp0f5h11.jpg?width=320&crop=smart&auto=webp&s=6267a3ae11e616453e129c58ae34635b98c4ab37: HTTP Error 404: Not Found
Failed to load image from URL https://preview.redd.it/ri7ut2wn8kv01.jpg?width=320&crop=smart&auto=webp&s=d44a419480c2073a2f0f3fa5f3701b308271331d: HTTP Error 404: Not Found
Failed to load image from URL https://preview.redd.it/5lddt3c142j21.jpg?width=320&crop=smart&auto=webp&s=4760c84252c012a0a940b81cac78bf5f05864b35: HTTP Error 404: Not Found
Failed to load image from URL nan: expected string or bytes-like object
Failed to load image from URL https://preview.redd.it/1j2wcot1s9g31.jpg?width=320&crop=smart&auto=webp&s=6814e8f1bc95d5f4b7e311b9afd06ef95fae8674: HTTP Error 404: Not Found
Failed to load image from URL https://preview.redd.it/q3kyrhxe5j431.jpg?width=320&crop=smart&auto=webp&s=3529dc31e753c462ae912aaab7c79ca5835c4ed6: HTTP Error 404: Not Found
Failed to load image from URL https://preview.redd.it/y63b4jazo

In [6]:
image_data.shape

(500, 224, 224, 3)

In [7]:
# Import required libraries
from transformers import RobertaModel, RobertaConfig
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader

# Load pre-trained Roberta model
roberta_config = RobertaConfig.from_pretrained('roberta-base')
roberta_model = RobertaModel.from_pretrained('roberta-base', config=roberta_config)

# Load pre-trained ResNet model
#resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Create a combined model with Roberta and ResNet
# ... code for multimodal fusion ...

# You can now use 'roberta_model' and 'resnet_model' as feature extractors to process text and image data, respectively, for multimodal fusion and fake tweet detection.

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Load Roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize text data
tokenized_text = [tokenizer.encode(str(text), max_length=50, pad_to_max_length=True) for text in padded_text]

# Convert tokenized text data to torch tensors
tokenized_text = torch.tensor(tokenized_text)

# Load pre-trained Roberta model for sequence classification
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(tokenized_text, fakeddit['2_way_label'], test_size=0.2, random_state=42)

# Convert labels to lists and then to torch tensors
y_train = torch.tensor(y_train.tolist())
y_test = torch.tensor(y_test.tolist())

# Create DataLoader for batching
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# Create DataLoader for batching the test set
test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define hyperparameters
num_epochs = 5
learning_rate = 2e-5

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(roberta_model.parameters(), lr=learning_rate)

# Move the model to GPU if available
if torch.cuda.is_available():
    roberta_model.cuda()

# Training loop
for epoch in range(num_epochs):
    roberta_model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        inputs = inputs.to('cuda') if torch.cuda.is_available() else inputs
        labels = labels.to('cuda') if torch.cuda.is_available() else labels
        outputs = roberta_model(inputs)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
    torch.cuda.empty_cache()  # Free up GPU memory

# Evaluation loop
roberta_model.eval()
with torch.no_grad():
    inputs = X_test.to('cuda') if torch.cuda.is_available() else X_test
    labels = y_test.to('cuda') if torch.cuda.is_available() else y_test
    outputs = roberta_model(inputs)
    predicted_labels = torch.argmax(outputs.logits, dim=1)
    num_correct = torch.sum(predicted_labels == labels)
    accuracy = num_correct.item() / len(labels)
    print(f'Accuracy: {accuracy:.4f}')


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSeque

Accuracy: 0.6800


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.model_selection import train_test_split

# Define ResNet18 model
class ResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet18, self).__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        self.resnet18.fc = nn.Linear(self.resnet18.fc.in_features, num_classes)
        
    def forward(self, x):
        x = self.resnet18(x)
        return x

# Instantiate ResNet18 model
model = ResNet18()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_data, fakeddit['2_way_label'], test_size=0.2)

# Reshape input data to have the correct shape
X_train = X_train.transpose((0, 3, 1, 2))
X_test = X_test.transpose((0, 3, 1, 2))

# Define data loaders
batch_size = 32
train_data = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train.values).long())
test_data = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test.values).long())

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Train model
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Evaluate model on test set
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        accuracy = 100 * correct / total
        
    print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch+1, num_epochs, loss.item(), accuracy))

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 99.8MB/s]


Epoch [1/10], Loss: 0.6242, Test Accuracy: 60.00%


In [ ]:
class ResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet18, self).__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(self.resnet18.children())[:-1])  # remove last fully connected layer
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return x


In [ ]:
def extract_features(data_loader, model):
    all_features = []
    all_labels = []
    
    # Set the model to evaluation mode
    model.eval()

    with torch.no_grad():
        for images, labels in data_loader:
            # Extract features from the last convolutional layer before the fully connected layer
            features = model(images)

            # Add the features and labels to lists
            all_features.append(features)
            all_labels.append(labels)

    # Concatenate the features and labels into tensors
    all_features = torch.cat(all_features)
    all_labels = torch.cat(all_labels)

    return all_features, all_labels

# Extract features from the training set
resnet_train_features, train_labels = extract_features(train_loader, model)

# Extract features from the test set
resnet_test_features, test_labels = extract_features(test_loader, model)


In [ ]:
#extracting roberta_features
def extract_features1(data_loader, model):
    model.eval()
    all_features = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to('cuda') if torch.cuda.is_available() else inputs
            labels = labels.to('cuda') if torch.cuda.is_available() else labels
            outputs = model(inputs)
            hidden_states = outputs.hidden_states
            if hidden_states is not None:
                pooled_output = hidden_states[-1][:, 0]
                all_features.append(pooled_output)
                all_labels.append(labels)

    if len(all_features) > 0:
        # Concatenate the features and labels into tensors
        all_features = torch.cat(all_features)
        all_labels = torch.cat(all_labels)
    else:
        # If no features extracted, return None for both features and labels
        all_features = None
        all_labels = None

    return all_features, all_labels


In [ ]:
roberta_features, _ = extract_features1(train_dataloader, roberta_model)  # replace train_dataloader and roberta_model with your own data loader and model

if resnet_train_features is not None and roberta_features is not None:
    # Convert Roberta features to NumPy array
    roberta_features = roberta_features.numpy()

    # Concatenate ResNet18 and Roberta features
    concatenated_features = np.concatenate((resnet_train_features, roberta_features), axis=1)
else:
    # If either feature is None, use only ResNet18 features
    concatenated_features = resnet_train_features



In [ ]:
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(concatenated_features, fakeddit['2_way_label'][:400], test_size=0.2, random_state=42)


# Convert data to PyTorch tensors
X_train = X_train.float()
y_train = torch.from_numpy(y_train.values).long()
X_test = X_test.float()
y_test = torch.from_numpy(y_test.values).long()

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 1e-4

# Define neural network architecture
class MultimodalModel(nn.Module):
    def __init__(self):
        super(MultimodalModel, self).__init__()
        self.fc1 = nn.Linear(1024+768, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=1)  # Concatenate feature vectors along dim=1
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Create DataLoader for batching
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Instantiate the model and move it to GPU if available
multimodal_model = MultimodalModel()
if torch.cuda.is_available():
    multimodal_model.cuda()

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(multimodal_model.parameters(), lr=learning_rate)






In [ ]:
# Training loop
for epoch in range(num_epochs):
    multimodal_model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        batch_size = inputs.size(0)  # Get the batch size
        inputs1 = inputs[:, :2048].view(batch_size, -1)  # Reshape to (batch_size, 2048)
        inputs2 = inputs[:, 2048:].view(batch_size, -1)  # Reshape to (batch_size, 2048)
        inputs1 = inputs1.to('cuda') if torch.cuda.is_available() else inputs1
        inputs2 = inputs2.to('cuda') if torch.cuda.is_available() else inputs2
        labels = labels.to('cuda') if torch.cuda.is_available() else labels
        outputs = multimodal_model(inputs1, inputs2)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
    torch.cuda.empty_cache()

# Evaluation loop
multimodal_model.eval()
with torch.no_grad():
    batch_size = X_test.size(0)  # Get the batch size
    inputs1 = X_test[:, :2048].view(batch_size, -1).to('cuda') if torch.cuda.is_available() else X_test[:, :2048].view(batch_size, -1)
    inputs2 = X_test[:, 2048:].view(batch_size, -1).to('cuda') if torch.cuda.is_available() else X_test[:, 2048:].view(batch_size, -1)
    labels = y_test.to('cuda') if torch.cuda.is_available() else y_test
    outputs = multimodal_model(inputs1, inputs2)
    predicted_labels = torch.argmax(outputs, dim=1)
    num_correct = torch.sum(predicted_labels == labels).item()
